In [ ]:
import tensorflow as tf
import os
import json
import skimage.io
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn.model import utils
from typing import Optional, Dict, List, Any
import numpy as np
import keras.engine.topology as KE


# Set paths
ROOT_DIR: str = os.getcwd()
DATA_DIR: str = os.path.join(ROOT_DIR, 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete')
MODEL_DIR: str = os.path.join(ROOT_DIR, 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\final_models')

# Crack dataset class
class CrackDataset(utils.Dataset):
    def __init__(self):
        super().__init__()
        self.bbox_data: Optional[Dict[str, Any]] = None  # Define bbox_data as an instance variable

    def load_data(self) -> Dict[str, str]:
        IMAGES_DIR: str = 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\Images'
        MASKS_DIR: str = 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\Masks'
        ANNOTATIONS_FILE: str = 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\BoundingBoxs'

        # Load image files
        images: List[str] = os.listdir(os.path.join(DATA_DIR, IMAGES_DIR))

        # Load mask files
        masks: List[str] = os.listdir(os.path.join(DATA_DIR, MASKS_DIR))

        # Load annotation data if the file exists
        annotations_file_path = os.path.join(DATA_DIR, ANNOTATIONS_FILE)
        
        if os.path.exists(annotations_file_path):
          raise FileNotFoundError(f"Annotations file '{ANNOTATIONS_FILE}' not found.")

        else:
            with open(annotations_file_path) as f:
                self.bbox_data = json.load(f)
            
            # Match images and masks
            images_dict: Dict[str, str] = dict(zip(images, masks))
            return images_dict

    def load_image(self, image_id: str) -> np.ndarray:
        image_path: str = os.path.join(DATA_DIR, 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\Images', image_id)
        image: np.ndarray = skimage.io.imread(image_path)
        return image

    def load_mask(self, image_id: str) -> np.ndarray:
        mask_path: str = os.path.join(DATA_DIR, 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\Masks', image_id)
        mask: np.ndarray = skimage.io.imread(mask_path)
        return mask

    def image_reference(self, image_id: str) -> str:
        return 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\Concrete\\Images' + image_id
    
def bbox_reference(self, image_id: str) -> Optional[Dict[str, Any]]:
    if self.bbox_data is not None:
        for bb in self.bbox_data:
            if bb['image_id'] == image_id:
                return bb  # Return the matching dictionary

    return None  # Return None if no matching 'image_id' is found

# Crack config
class CrackConfig(Config):
    # Hyperparameters
    NAME: str = "crack"  # Name of your configuration
    NUM_CLASSES: int = 1 + 1  # Crack + Background
    GPU_COUNT: int = 1
    IMAGES_PER_GPU: int = 1  # You can adjust this based on your GPU memory

# Create crack model
dataset = CrackDataset()
model = modellib.MaskRCNN(
    mode='training',
    config=CrackConfig(),
    model_dir=MODEL_DIR
)

# Train weights on the crack dataset
images_dict = dataset.load_data()  # Load the data
images, masks = list(images_dict.keys()), list(images_dict.values())
model.train(
    images,
    masks,
    learning_rate=0.001,
    epochs=50,
    layers='all'
)

# Save trained crack model
model.keras_model.save_weights(os.path.join(MODEL_DIR, 'C:\\New folder\\Dr. Surya\\MaskRCNN\\Unity_Generation\\crack_model.h5'))

# Evaluate on the validation dataset
val_dataset = CrackDataset()
val_images_dict = val_dataset.load_data()  # Load validation data
val_images, val_masks = list(val_images_dict.keys()), list(val_images_dict.values())
scores = model.keras_model.evaluate(val_images, val_masks)

# Predict crack on a test image (replace 'test_img' with your image data)
test_img = None  # Load your test image here
results = model.detect([test_img], verbose=1)

: 